# Reserach data 
### Additional Dataset Information

The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be ('yes') or not ('no') subscribed. 

There are four datasets: 
1) bank-additional-full.csv with all examples (41188) and 20 inputs, ordered by date (from May 2008 to November 2010), very close to the data analyzed in [Moro et al., 2014]
2) bank-additional.csv with 10% of the examples (4119), randomly selected from 1), and 20 inputs.
3) bank-full.csv with all examples and 17 inputs, ordered by date (older version of this dataset with less inputs). 
4) bank.csv with 10% of the examples and 17 inputs, randomly selected from 3 (older version of this dataset with less inputs). 
The smallest datasets are provided to test more computationally demanding machine learning algorithms (e.g., SVM). 

The classification goal is to predict if the client will subscribe (yes/no) a term deposit (variable y).

### Import Data

In [4]:
import pandas as pd

bank_data = pd.read_csv("Data/bank-additional/bank-additional/bank-additional.csv", delimiter=";") #all examples 100%

bank_data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,30,blue-collar,married,basic.9y,no,yes,no,cellular,may,fri,...,2,999,0,nonexistent,-1.8,92.893,-46.2,1.313,5099.1,no
1,39,services,single,high.school,no,no,no,telephone,may,fri,...,4,999,0,nonexistent,1.1,93.994,-36.4,4.855,5191.0,no
2,25,services,married,high.school,no,yes,no,telephone,jun,wed,...,1,999,0,nonexistent,1.4,94.465,-41.8,4.962,5228.1,no
3,38,services,married,basic.9y,no,unknown,unknown,telephone,jun,fri,...,3,999,0,nonexistent,1.4,94.465,-41.8,4.959,5228.1,no
4,47,admin.,married,university.degree,no,yes,no,cellular,nov,mon,...,1,999,0,nonexistent,-0.1,93.200,-42.0,4.191,5195.8,no


### Data Prep

In [5]:
# Preprocessing

from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.discriminant_analysis import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline

# PP target
bank_target = bank_data['y']
bank_preprocessed = bank_data.drop(columns='y')

label_encoder = preprocessing.LabelEncoder()
bank_target = label_encoder.fit_transform(bank_target) # encoded in 0 / 1


preprocessor = preprocessing.OneHotEncoder()
estimator = KNeighborsClassifier(n_neighbors=4)
# preprocessor = ColumnTransformer(
#     transformers=[
#         # use StandardScaler for Temperature and Humidity
#         ('scaler', StandardScaler(), ['Temperature', 'Humidity']),
#         # use OneHotEncoder for Outlook and Wind
#         ('encoder', OneHotEncoder(), ['Outlook', 'Wind']) ])

# encoded = pd.DataFrame(data_encoder.fit_transform(bank_preprocessed[['job','marital','education','default','housing','loan','contact','month','day_of_week','poutcome']]).toarray(),columns=encoder.get_feature_names_out(['job','marital','education','default','housing','loan','contact','month','day_of_week','poutcome']))

# bank_preprocessed = bank_preprocessed.drop(columns=['job','marital','education','default','housing','loan','contact','month','day_of_week','poutcome'])
# bank_preprocessed = pd.concat([bank_preprocessed,encoded],axis=1)
# bank_preprocessed.head()

pipeline = Pipeline([('preprocessing', preprocessor), ('classification', estimator)])


### First Trys

In [6]:
#General approach with Model evaluation.. Recall or Precision?
#F1 & accuracy always, we can include a cost example for r & p
#r: how many potential customers to I miss (which were actually using the account)
#p: What percentage of the selected customers actually use an account like that
from sklearn.model_selection import cross_val_score


accuracy = cross_val_score(pipeline, bank_data, bank_target, cv=10, scoring='accuracy')

print("Average Accuracy = {}%".format(accuracy.mean() * 100.0))

ValueError: 
All the 10 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\I568830\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\I568830\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\I568830\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\pipeline.py", line 423, in fit
    Xt = self._fit(X, y, **fit_params_steps)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c: